In [1]:
import findspark
from pyspark.sql import SparkSession

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/07/17 09:33:43 WARN Utils: Your hostname, RNTDELL000700 resolves to a loopback address: 127.0.1.1; using 192.168.1.131 instead (on interface wlp0s20f3)
23/07/17 09:33:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/17 09:33:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Transformaciones en un RDD

Los RDDs son inmutables y cada operación crea un nuevo RDD. Las dos operaciones principales que se pueden realizar en una red son transformaciones y acciones:

    -**Transformaciones:** cambian los elementos en el RDD. Algunos ejemplos pueden ser dividir el elemento de entrada, filtrar los elementos y realizar cálculos de algún tipo. Varias transformaciones se pueden realizar en una secuencia, sin embargo, no se lleva a cabo ninguna ejecución durante la planificación. Para las transformaciones spark las agrega al DAC de calculo. DAG proviene de Directed Acyclic Graph y sólo cuando el controlador solicita algunos datos de este DAG realmente se ejecuta; a esto se le llama evaluación perezosa o lazy evaluation.


#### Tipos de transformaciones

Las transformaciones se pueden dividir en cuatro categorías:

    -**Transformaciones generales:** son funciones de transformación que manejan la mayoría de los casos de uso de propósito general. Estas aplican la lógica de transformaciones a los RDD existentes y generan un nuevo RDD de las operaciones comunes de agregación, filtros, etc. Ejemplos: map, filter, flatMap, groupByKey, sortByKey, combineByKey. 
    -**Transformaciones matemáticas o estadísticas:** son funciones de transformación que manejan alguna funcionalidad estadística y que generalmente aplican alguna operación matemática o estadística en el RDD existente, generando un nuevo RDD. El muestreo es un gran ejemplo de esto y se usa a menudo en las aplicaciones de spark. Ejemplos: sampleByKey, randomSplit.
    -**Transformaciones de conjunto o relacionales:** son funciones de que manejan transformaciones como uniones de conjuntos de datos y otras funciones algebraicas relacionales como grupo. Funcionan aplicando la lógica de transformación a los RD existentes y generando un nuevo RDD. Ejemplos: cogroup, join, subtractByKey, fullOuterJoin, leftOuterJoin, rightOuterJoin.
    -**Transformaciones basadas en estructura de datos:** son funciones de transformación que operan en las estructuras de datos subyacentes del RDD, como las particiones por ejemplo. Estas funciones pueden trabajar directamente en particiones sin tocar directamente los elementos o los datos dentro del RDD. Por lo general, las mejoras de rendimiento se pueden realizar redistribuyendo las particiones de datos de acuerdo con el estado del plotter, el tamaño de los datos y los requisitos exactos del caso de uso. Ejemplos: partitionBy, repartition, zipwithIndex, coalesce.